<a href="https://www.kaggle.com/code/sakham/pytorch-lob?scriptVersionId=116672504" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder   
import torch
import torch.nn  as nn

In [2]:
traindata = '/kaggle/input/limit-orderbook-data/Train_Dst_NoAuction_DecPre_CF_7.txt'
cvdata = '/kaggle/input/limit-orderbook-data/Test_Dst_NoAuction_DecPre_CF_9.txt'                                                              

In [3]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
BATCH = 128

"""
    DataLoader file fot the LOB dataset gives out the lob matrix of size (batch_size, 1, Seq_len, 4*levels)
    Data can is taken from FI-2010 Dataset with default horizon prediction level = 50 (147, columns)
    Horizon = 10 (Level = 144) 
    For custom Lobster Data set we we change the horizon according to the dataset that we have defined  
"""


class GenMatData(Dataset):
    def __init__(self, path, level=10, horizon=147, sequencelen=100):
        metadata = np.loadtxt(path).T
        self.levels = level
        self.sequence_len = sequencelen
        self.train_data = metadata[:, :4*level]
        self.predictions = metadata[:, horizon]

    def __len__(self):
        return self.train_data.shape[0] - self.sequence_len + 1

    def __getitem__(self, item):
        lob_instance = torch.tensor(self.train_data[item:item+self.sequence_len, :]).float()
        out = torch.tensor(self.predictions[item+self.sequence_len-1])
        return lob_instance, out-1


def LobDataLoader(batch, lobpath, shuf, level=10, seq_len=100, Horizon=147):
    dataset = GenMatData(path=lobpath, level=level, horizon=Horizon, sequencelen=seq_len)
    in_loader = DataLoader(dataset, shuffle=shuf, batch_size=batch, pin_memory=True)
    return in_loader


In [4]:
# train_loader = LobDataLoader(batch=BATCH, lobpath=traindata, shuf=True)
# cv_loader = LobDataLoader(batch=BATCH, lobpath=cvdata, shuf=True)                                                                           

In [5]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=4):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding='same', bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

In [6]:
x = torch.randn(32, 10, 100, 40)
mod = SpatialAttention()
out = mod(x)
out = out*x
out.shape

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/conv.py:440: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/Convolution.cpp:659.)
  self.padding, self.dilation, self.groups)


torch.Size([32, 10, 100, 40])

In [7]:
x = torch.randn(32, 1, 100, 40)
op = torch.max(x, dim=1, keepdim=True)

In [8]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

In [9]:
m = nn.AdaptiveAvgPool2d((1))
input = torch.randn(32, 10, 8, 9)
output = m(input)
output.size()

torch.Size([32, 10, 1, 1])

In [10]:
import torch
import torch.nn as nn

class BaseCNN(nn.Module):
    def __init__(self, in_chanel, out_chanel, kernel, strides):
        super(BaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_chanel, out_chanel, kernel, bias=False, stride=strides)
        self.norm1 = nn.BatchNorm2d(out_chanel)
        self.norm2 = nn.BatchNorm2d(out_chanel)
        self.norm3 = nn.BatchNorm2d(out_chanel)
        self.conv2 = nn.Conv2d(out_chanel, out_chanel, kernel_size=(3, 1), padding='same')
        self.conv3 = nn.Conv2d(out_chanel, out_chanel, kernel_size=(3, 1), padding='same')
        self.l_relu = nn.LeakyReLU(negative_slope=3e-2, inplace=True)

    def forward(self, mat):
        out = self.l_relu(self.norm1(self.conv1(mat)))
        out = self.l_relu(self.norm2(self.conv2(out)))
        out = self.l_relu(self.norm3(self.conv3(out)))
        return out


class DynamicConv(nn.Module):
    def __init__(self, in_chanel, out_chanel, dims):
        super(DynamicConv, self).__init__()
        """
                We can initialize the linear layer of the model in one other way by using 
                nn.adaptiveavgpool2d from nn and thus compute the attention in that manner 
                reducing the model weight. 
                in_chanel : input chanel in dynamic conv layer 
                out_chanel : number of output chanel 
                dim : dimension (L*W) of the input matrix 
        """
        self.dim = dims
        self.dconv1 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(1, self.dim[1]), padding='same', bias=False)
        self.dconv2 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(5, self.dim[1]), padding='same', bias=False)
        self.dconv3 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(3, self.dim[1]), padding='same', bias=False)
        self.linear = nn.Linear(self.dim[0]*self.dim[1]*in_chanel, 3)
        self.leaky = nn.LeakyReLU(negative_slope=1e-2, inplace=True)
        self.bnorm1 = nn.BatchNorm2d(out_chanel)
        self.bnorm2 = nn.BatchNorm2d(out_chanel)
        self.bnorm3 = nn.BatchNorm2d(out_chanel)
        self.soft = nn.Softmax(dim=1)

    def forward(self, mat):
        weight = self.soft(self.linear(mat.view((mat.size(0), -1))))
        out1 = self.leaky(self.bnorm1(self.dconv1(mat))).unsqueeze(1)
        out2 = self.leaky(self.bnorm2(self.dconv2(mat))).unsqueeze(1)
        out3 = self.leaky(self.bnorm3(self.dconv3(mat))).unsqueeze(1)
        out = torch.cat((out1, out2, out3), dim=1)
        out = torch.sum(torch.mul(out, weight.reshape(weight.size(0), weight.size(1), 1, 1, 1)), dim=1)

        return out


class DeepLob(nn.Module):
    def __init__(self, in_chanels, hidden_size, num_layers, seq_length):
        super(DeepLob, self).__init__()
        self.one = BaseCNN(in_chanel=in_chanels, out_chanel=32, kernel=(1, 2), strides=(1, 2))
        self.two = BaseCNN(in_chanel=32, out_chanel=64, kernel=(1, 2), strides=(1, 2))
        self.three = BaseCNN(in_chanel=64, out_chanel=128, kernel=(1, 10), strides=(1, 10))
        self.dyconv = DynamicConv(in_chanel=128, out_chanel=128, dims=(100, 1))
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.liner = nn.Sequential(nn.Linear(seq_length*hidden_size, hidden_size), nn.ReLU(), # Just giving last LSTM node to Dense UNIT
                                   nn.Linear(hidden_size, 3))

    def forward(self, mat):
        out_lob = self.one(mat)
        out_lob = self.two(out_lob)
        out_lob = self.three(out_lob)
        out_lob = self.dyconv(out_lob)
        out_lob = out_lob.squeeze(-1)
        out_lob = torch.transpose(out_lob, 1, 2)
        out_lob, (ho, co) = self.lstm(out_lob)
        out_lob = out_lob.reshape(out_lob.size(0), -1)
        return self.liner(out_lob)

In [11]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dims = self.embed_size // self.heads
        assert (self.head_dims * heads == self.embed_size)
        self.keys = nn.Linear(self.head_dims, self.head_dims, bias=True)
        self.value = nn.Linear(self.head_dims, self.head_dims, bias=True)
        self.query = nn.Linear(self.head_dims, self.head_dims, bias=True)
        self.fc_out = nn.Linear(heads*self.head_dims, embed_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, k, q, v, mask):

        n = k.shape[0]
        key_len, query_len, value_len = k.shape[1], q.shape[1], v.shape[1]

        k = k.reshape(n,   key_len, self.heads, self.head_dims)
        q = q.reshape(n, query_len, self.heads, self.head_dims)
        v = v.reshape(n, value_len, self.heads, self.head_dims)

        k = self.keys(k)
        q = self.query(q)
        v = self.value(v)

        # query (N, query_len, heads, head_dim)
        # keys  (N, keys_len , heads, head_dim )
        # value (N, value_len, heads, head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [q, k])
        # energy (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e12"))
        attention = torch.softmax(energy/self.embed_size**(1/2), dim=3)
        out = torch.einsum("nhqk,nkhd->nqhd", [attention, v]).reshape(n, query_len, self.heads*self.head_dims)
        out = self.fc_out(out)
        return out


class Transformer(nn.Module):
    def __init__(self, embed_size, heads):
        super(Transformer, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.attention = SelfAttention(embed_size, heads)
        self.layern = nn.LayerNorm(embed_size)
        self.fc = nn.Linear(embed_size, embed_size)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, inputs):
        n = inputs.shape[1]
        batch = inputs.shape[0]
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        mask = torch.tril(torch.ones(n, n)).expand(batch, 1, n, n).to(device)
#         mask = None
        out = self.layern(self.attention(inputs, inputs, inputs, mask) + inputs)
        out = self.layern(self.activation(self.fc(out)) + out)
        return out


class Model(nn.Module):
    def __init__(self, encoding_dim=4, embed_dim=40, seq_len=100):
        super(Model, self).__init__()
        self.seq_len = seq_len
        self.trans1 = nn.MultiheadAttention(embed_dim + encoding_dim, 11,batch_first=True)
        self.trans2 = nn.MultiheadAttention(embed_dim + encoding_dim, 11,batch_first=True)
        self.MLP = nn.Sequential(
            nn.Linear(seq_len*(embed_dim+encoding_dim), embed_dim),
            nn.ReLU())
        self.MLP2 = nn.Sequential(
            nn.Linear(embed_dim, embed_dim//5),
            nn.Softmax(dim=-1),
            nn.Dropout(p=0.2),
            nn.Linear(embed_dim//5, 3),
            nn.Softmax(dim=-1)
        )
    def position_encoding(self, seq_len: int, dim_model: int, device: torch.device = torch.device("cuda")):
        pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)
        dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)
        phase = pos / (1e4 ** (torch.div(dim, dim_model, rounding_mode='trunc')))
        return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

    def forward(self, x):
        batch = x.shape[0]
        sl = x.shape[1]
        dim = 4
        resul = self.position_encoding(seq_len=sl, dim_model=dim, device=torch.device("cuda"))
        resul = resul.expand(batch, sl, dim)
        x = torch.cat((x, resul), dim=-1)
        n = self.seq_len
#         mask = torch.tril(torch.ones(n, n)).expand(batch, 1, n, n).to(device)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        mask = torch.tril(torch.ones(n, n)).to(device)
        mask = mask.to(device)
    
        out,i = self.trans1(x,x,x, attn_mask=mask)
        out,i = self.trans2(out,out,out, attn_mask=mask)
        out = out.reshape(batch, -1)
        out = self.MLP(out)
        out = self.MLP2(out)
        return out

In [12]:
class TSM(nn.Module):
    def __init__(self, inc, outc, ker, shift):
        super(TSM, self).__init__()
        self.shift = shift
        self.conv1 = nn.Conv2d(in_channels=inc, out_channels=outc,  kernel_size=ker, bias=False, stride=(1,1), padding='same')
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(outc)
        self.convi = nn.Conv2d(in_channels=inc, out_channels=outc,  kernel_size=ker, bias=False, stride=(1,1), padding='same')
        self.bnormi = nn.BatchNorm2d(inc)
        
    def forward(self, x):
        x = self.activation(self.bnormi(self.convi(x)))
        out = torch.zeros_like(x)
        out[:,:, 1:, -self.shift:] = x[:,:,:-1,-self.shift:]
        out[:,:,:,:-self.shift] = x[:,:,:,:-self.shift]
        out = self.activation(self.bnorm(self.conv1(out)))
        batch, c, h, w  = out.size()
        return out + x.expand(batch, c, h, w)

In [13]:
class ModelC(nn.Module):
    def __init__(self):
        super(ModelC, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(1, 2), bias=False, stride=(1,2))
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8,  kernel_size=(1, 2), bias=False, stride=(1,1))
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=8,  kernel_size=(1, 4), bias=False, stride=(1,1))
        self.sh = TSM(inc=8, outc=8, ker=(1,2), shift=4)
#         self.conv3 = nn.Conv2d(in_channels=4, out_channels=8,  kernel_size=(1, 2), bias=False, stride=(1,1), dilation=4)
#         self.conv4 = nn.Conv2d(in_channels=8, out_channels=10,  kernel_size=(1, 2), bias=False, stride=(1,1), dilation=8)
#         self.conv5 = nn.Conv2d(in_channels=10, out_channels=10,  kernel_size=(1, 2), bias=False, stride=(1,1), dilation=16)
        self.activation = nn.LeakyReLU(negative_slope=3e-2, inplace=True)
        self.norm1 = nn.BatchNorm2d(4)
        self.norm2 = nn.BatchNorm2d(8)
        self.norm3 = nn.BatchNorm2d(8)
#         self.norm3 = nn.BatchNorm2d(8)
#         self.norm4 = nn.BatchNorm2d(10)
#         self.norm5 = nn.BatchNorm2d(10)()

        self.sa = SpatialAttention()
        self.ca = ChannelAttention(in_planes=8, ratio=2)
        
        self.sa2 = SpatialAttention()
        self.ca2 = ChannelAttention(in_planes=8, ratio=4)
        
        self
        
        
    def forward(self, inputs):
        out = self.activation(self.norm1(self.conv1(inputs)))
        out = self.activation(self.norm2(self.conv2(out)))
        out = out * self.ca2(out)
        out = out * self.sa2(out)
        out = self.sh(out)
        out = self.activation(self.norm3(self.conv3(out)))
        out = out * self.ca(out)
        out = out * self.sa(out)
#         out = self.activation(self.norm3(self.conv3(out)))
#         out = self.activation(self.norm4(self.conv4(out)))
#         out = self.activation(self.norm5(self.conv5(out)))
        return out

    
#     // batchsize, seq_len, 100 + 4
class ModelNet(nn.Module):
    def __init__(self, embed_dim, encoding_dim, seq_len, device):
        super(ModelNet, self).__init__()
        self.conv = ModelC()
        self.device = device
        self.seq_len = seq_len
        self.encoding_dim = encoding_dim
        self.MLP = nn.Sequential(
            nn.Linear(seq_len * (8*embed_dim), embed_dim),
            nn.ReLU(),
#             nn.Dropout(p=0.3),
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(),
#             nn.Dropout(p=0.3),
            nn.Linear(embed_dim // 2, 3),
            nn.Softmax(dim=-1)
            )
    def position_encoding(self, seq_len: int, dim_model: int, device: torch.device = torch.device("cpu")):
        pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)
        dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)
        phase = pos / (1e4 ** (torch.div(dim, dim_model, rounding_mode='trunc')))
        return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

    def forward(self, x):
        batch = x.shape[0]
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.transpose(1,2)
        x = x.reshape(batch, self.seq_len, -1)
        sl = x.shape[1]
#         resul = self.position_encoding(seq_len=sl, dim_model=self.encoding_dim, device=self.device)
#         resul = resul.expand(batch, sl, self.encoding_dim)
#         x = torch.cat((x, resul), dim=-1)
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#         n = self.seq_len
#         mask = torch.tril(torch.ones(n, n)).expand(batch, 1, n, n).to(device)
#         mask = torch.tril(torch.ones(n, n)).to(device)
#         out,o = self.trans1(x,x,x, attn_mask=mask)
#         out,o = self.trans2(out,out,out, attn_mask=mask)
        out = x.reshape(batch, -1)
        out = self.MLP(out)
        return out
        

In [14]:
class ModelT(nn.Module):
    def __init__(self):
        super(ModelT, self).__init__()
        self.convb1 = nn.Conv2d(in_channels=1, out_channels=2,  kernel_size=(1,2), bias=False, stride=(1,2))        
        self.tsm1 = TSM(inc=2, outc=2, ker=(1,2), shift=4)
        self.convb2 = nn.Conv2d(in_channels=2, out_channels=2,  kernel_size=(1,2), bias=False, stride=(1,2))
        self.tsm2 = TSM(inc=2, outc=2, ker=(1,2), shift=2)
        self.convb3 = nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(1,4), stride=(1,1), bias=False)
        self.activ = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(2)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(2)
        
        self.dense = nn.Sequential(nn.Linear(2*100*7, 2*100),
                                  nn.LeakyReLU(0.01), 
                                  nn.Dropout(p=0.3),
                                  nn.Linear(2*100, 64),
                                  nn.LeakyReLU(0.01), 
                                  nn.Dropout(p=0.3),
                                  nn.Linear(64, 3),
                                  nn.Softmax(dim=-1))
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.activ(self.bn1(self.convb1(x)))
        out = self.tsm1(out)
        out = self.activ(self.bn2(self.convb2(out)))
        out = self.tsm2(out)
        out = self.activ(self.bn3(self.convb3(out)))
        batch = out.size()[0]
        out = out.view(batch, -1)
        out = self.dense(out)
        return out
                                
                                

In [15]:
# batch, 16, 100, 10
(10 - (4-1) -1 )/1 + 1                                                                                                                        

7.0

In [16]:
x = torch.randn(32, 1, 100, 40)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = x.to(device)
mod = ModelC()
mod = mod.to(device)
mod(x).shape

torch.Size([32, 8, 100, 16])

In [17]:
from sklearn.metrics import classification_report
def accu(model, loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    accuracy = []
    model.eval()
    with torch.no_grad():
        for mat, label in loader:
            mat = mat.to(device)
            label = label.to(device)
            pre = model(mat)
            _, pre = torch.max(pre, dim=1)
            accuracy.append((pre==label).sum()/label.size(0))
        accuracy = torch.tensor(accuracy).mean()
        return accuracy.item()

In [18]:
import torch.optim as optim
from tqdm import trange, tqdm
def train():
    EPOCHS = 200
    traindata = '/kaggle/input/limit-orderbook-data/Train_Dst_NoAuction_DecPre_CF_7.txt'
    cvdata = '/kaggle/input/limit-orderbook-data/Test_Dst_NoAuction_DecPre_CF_9.txt'    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    loader = LobDataLoader(batch=32, lobpath=traindata, shuf=True)
    cvloader = LobDataLoader(batch=32, lobpath=cvdata, shuf=True)
    model = ModelNet(encoding_dim=5, embed_dim=16 , seq_len=100, device = device)
#     model = ModelT()
    model = model.to(device)
    opt = optim.Adam(params=model.parameters(), lr=1e-4)
    crit = nn.CrossEntropyLoss()
    for epoch in range(EPOCHS):
        print('EPOCH {}'.format(epoch))
        for data, labels in tqdm(loader):
            data = data.to(device)
            labels = labels.to(device)
            prediction = model(data)
#             print(data.shape)
#             print(prediction.shape)
#             print(labels.shape)
            loss = crit(prediction, labels.long())
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        print('Epoch {} training Accuracy {} CV accuracy {}'.format(epoch, accu(model, loader), accu(model, cvloader)))
        model.train()            


In [19]:
train()

EPOCH 0


100%|██████████| 7958/7958 [01:18<00:00, 101.79it/s]


Epoch 0 training Accuracy 0.4438299536705017 CV accuracy 0.4414447247982025
EPOCH 1


100%|██████████| 7958/7958 [01:17<00:00, 102.84it/s]


Epoch 1 training Accuracy 0.4861868917942047 CV accuracy 0.44035595655441284
EPOCH 2


...

Epoch 197 training Accuracy 0.8264420628547668 CV accuracy 0.48621439933776855
EPOCH 198


100%|██████████| 7958/7958 [01:12<00:00, 109.02it/s]


Epoch 198 training Accuracy 0.8292797803878784 CV accuracy 0.49070143699645996
EPOCH 199


100%|██████████| 7958/7958 [01:13<00:00, 107.67it/s]


Epoch 199 training Accuracy 0.8333308696746826 CV accuracy 0.4889174997806549


In [20]:
class dila(nn.Module):
    def __init__(self):
        super(dila, self).__init__()
        self.convo = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(1, 2), bias=False, stride=(1,1), dilation = 4)
        self.norm = nn.BatchNorm2d(20)
        self.ac = nn.ReLU(inplace = True)
    def forward(self, x):
        return self.ac(self.norm(self.convo(x)))                                                                                              

In [21]:
trial = ModelC()
x = torch.ones(32,4,4,8)                                                                                                                      

In [22]:
xtemp = x                                                                                                                                     

In [23]:
nt, c, h, w = x.size()                                                                                                                        

In [24]:
 torch.tril(torch.ones(4, 4))

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])